# A2: Q4 and Q5

In [1]:
import numpy as np
from scipy.sparse import dok_matrix
from scipy.sparse import csc_matrix
from copy import deepcopy
import matplotlib.pyplot as plt

# A2Q4a: `SparseMatMult`

In [2]:
def SparseMatMult(G, x):
    '''
      y = SparseMatMult(G, x)
      
      Multiplies a vector (x) by a sparse matrix G,
      such that y = G @ x .
      
      Inputs:
        G is an NxM dictionary-of-keys (dok) sparse matrix
        x is an M-vector
      
      Output:
        y is an N-vector
    '''
    rows,cols = G.nonzero()
    Nrows,Ncols = np.shape(G)
    y = np.zeros(Nrows)
    
    # === YOUR CODE HERE ===
    C = [[0] * Ncols for i in range(Nrows)]
    for i in range(Nrows):
        for j in range(Ncols):
            C[i][j] = G[i,j]
    for i in range(Nrows):
        for j in range(Ncols):  
            y[i] += C[i][j] * x[j] 
    return y

In [3]:
# Simple test
#     [1  0  0]      [ 0.1 ]
# A = [0  0 -1]  b = [ 0.2 ]
#     [0  2  0]      [ 0.3 ]
A = dok_matrix((3,3), dtype=np.float32)
A[0,0] = 1.
A[1,2] = -1.
A[2,1] = 2.
b = np.array([0.1, 0.2, 0.3])
y = SparseMatMult(A, b)
print(f'y = {y}')
print(f'Answer should be [ 0.1 -0.3  0.4]')

y = [ 0.1 -0.3  0.4]
Answer should be [ 0.1 -0.3  0.4]


# A2Q4b: `PageRank`

In [4]:

def PageRank(G, alpha):
    '''
     p, iters = PageRank(G, alpha)

     Computes the Google Page-rank for the network in the adjacency matrix G.
     
     Note: This function never forms a full RxR matrix, where R is the number
           of node in the network.

     Input
       G     is an RxR adjacency matrix, G[i,j] = 1 iff node j projects to node i
             Note: G must be a dictionary-of-keys (dok) sparse matrix
       alpha is a scalar between 0 and 1

     Output
       p     is a probability vector containing the Page-rank of each node
       iters is the number of iterations used to achieve a change tolerance
             of 1e-8 (changes to elements of p are all smaller than 1e-8)
    '''
    R = np.shape(G)[0]
    rows,cols = G.nonzero()
    iters = 0

    # === YOUR CODE HERE ===
    colsum = np.zeros(R)
    
    zipped = []
    for m in range(len(rows)):
        zipped.append([rows[m],cols[m]])
        
    for r, c in zipped:
        colsum[c] += G[r,c]   
    
    
    # vector A
    A = G
    for r, c in zipped:
        A[r,c] /= colsum[c]    
   
    d=[]
    
    for s in colsum:
        if s==0:
            d.append(1.)
        else:
            d.append(0.)
    
    
    e = np.ones(R, dtype=float)
    
    # vector b
    b = np.ones(R, dtype=float) / R 
    
    
    error = 100
    
    
    while error > 0.00000001:
        
        
        bo = alpha*(SparseMatMult(A, b) + e*(d*b)/R) + (1-alpha)/R*e
        
        error = np.sum(abs(b-bo))

        b = bo
        
        iters = iters + 1
    
    p = b
    
    return p, iters



# A2Q5: Illegal Trading Network

## (a) Create sparse matrix

In [5]:
# === YOUR CODE HERE ===
A = dok_matrix((12,12), dtype=np.float32)
#A[0,0] = 1
A[0,1]=0.09
A[0,10]=0.47
A[0,11]=0.06

A[1,0]=0.24
#A[1,1]=1
A[1,2]=0.13
A[1,7]=0.06
A[1,8]=0.19
A[1,9]=0.42

A[2,1]=0.39
#A[2,2]=1
A[2,3]=0.24
A[2,5]=0.3
A[2,6]=0.21
A[2,7]=0.47

A[3,2]=0.23
#A[3,3]=1
A[3,4]=0.07
A[3,5]=0.05
A[3,6]=0.21

A[4,3]=0.2
#A[4,4]=1
A[4,5]=0.25
A[4,6]=0.18

A[5,2]=0.27
A[5,3]=0.32
A[5,4]=0.6
#A[5,5]=1
A[5,6]=0.1

A[6,2]=0.17
A[6,3]=0.24
A[6,4]=0.33
A[6,5]=0.4
#A[6,6]=1
A[6,7]=0.29
A[6,8]=0.22

A[7,1]=0.39
A[7,2]=0.2
A[7,6]=0.15
#A[7,7]=1
A[7,8]=0.22

A[8,1]=0.09
A[8,6]=0.15
A[8,7]=0.18
#A[8,8]=1
A[8,9]=0.5
A[8,11]=0.53

A[9,1]=0.04
A[9,8]=0.28
#A[9,9]=1
A[9,10]=0.24

A[10,0]=0.38
A[10,9]=0.08
#A[10,10]=1
A[10,11]=0.41

A[11,0]=0.38
A[11,8]=0.09
A[11,10]=0.29
#A[11,11]=1

b = np.array([1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12, 1/12])
y = SparseMatMult(A, b)
print(f'y = {y}')

y = [0.05166667 0.08666666 0.13416667 0.04666667 0.0525     0.1075
 0.1375     0.08       0.12083333 0.04666667 0.0725     0.06333333]


## (b) Run PageRank on network

In [6]:
# === YOUR CODE HERE ===
print(PageRank(A,alpha=1))

(array([0.01188404, 0.05658151, 0.17794871, 0.09460529, 0.09091633,
       0.15178567, 0.18916029, 0.10195167, 0.07236864, 0.02554492,
       0.01257685, 0.0146764 ]), 61)


## (c) Note to police

DOUBLE-CLICK TO PLACE YOUR COMMENTS HERE


In [7]:
#node H is the most influencial, as it has the highest probability.